# Bidirectional RNN (Bi-RNN) Implementation

This notebook demonstrates a Bidirectional Recurrent Neural Network for improved sequence understanding.

## Dataset
We'll use IMDB movie reviews dataset for sentiment analysis.

## Topics Covered:
1. Bidirectional Processing
2. Forward and Backward Context
3. Enhanced Sentiment Analysis
4. Comparison with Unidirectional RNN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print(f"TensorFlow Version: {tf.__version__}")

In [ ]:
# Load and preprocess data
vocab_size = 10000
max_length = 200

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
# Build Bidirectional RNN model
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_length),
    
    # Bidirectional RNN layers - processes sequence both ways
    Bidirectional(SimpleRNN(128, return_sequences=True)),
    Dropout(0.3),
    
    Bidirectional(SimpleRNN(64)),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    Dropout(0.3),
    
    Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=128,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)

In [ ]:
# Plot training history
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Bidirectional RNN - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Bidirectional RNN - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()

print("Test Accuracy:", np.mean(y_pred == y_test))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix - Bidirectional RNN')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

In [ ]:
# Save model
model.save('birnn_model.h5')
print("Model saved!")

## Summary

### Bidirectional RNN Advantages:
1. **Context from Both Directions**: Sees past and future context
2. **Better Understanding**: Improved comprehension of sequences
3. **Higher Accuracy**: Typically outperforms unidirectional RNN
4. **Complete Information**: Uses full sequence context

### When to Use Bi-RNN:
- When entire sequence is available (not real-time)
- Text classification where context matters
- Named entity recognition
- Speech recognition (offline)

### Bi-RNN vs RNN:
- **Bi-RNN**: Better accuracy, more parameters (2x)
- **RNN**: Faster, works for real-time applications
- Bi-RNN doubles the hidden units (forward + backward)